In [136]:
include("../utils.jl")
data = getdata("../data/7")
println(data[1:2])
function parsedata(d)
    m = match(r"Step ([A-Z]) must be finished before step ([A-Z]) can begin.", d)
    pre = m.captures[1][1]
    post = m.captures[2][1]
    return [pre, post]
end
data = map(parsedata, data)
println(data[1:2])

["Step A must be finished before step I can begin.", "Step M must be finished before step Q can begin."]
Array{Char,1}[['A', 'I'], ['M', 'Q']]


In [137]:
preconditions = Dict{Char,Array{Char,1}}()
for d in data
    pre, post = d[1], d[2]
    if post in keys(preconditions)
        push!(preconditions[post], pre)
    else
        preconditions[post] = [pre]
    end
    if !(pre in keys(preconditions))
        preconditions[pre] = Array{Char,1}()
    end
end
println(preconditions)
preconditions_save = deepcopy(preconditions);

Dict('E'=>['M', 'Y', 'G'],'Z'=>['H', 'F', 'S', 'N', 'P', 'V'],'X'=>['N', 'J', 'S', 'D', 'K', 'R', 'B'],'B'=>[],'C'=>['K'],'D'=>['B', 'Y', 'C', 'K'],'A'=>[],'R'=>['Y', 'S', 'T', 'F', 'L', 'P', 'Z', 'B'],'G'=>[],'N'=>['G', 'O', 'L', 'T', 'I', 'W', 'S', 'D'],'Q'=>['M', 'O', 'W', 'K', 'Y'],'M'=>['A'],'K'=>['B'],'P'=>['C', 'D', 'E', 'F', 'V', 'A'],'O'=>['E', 'A', 'C'],'I'=>['A', 'U', 'E', 'F', 'Y', 'V', 'H'],'F'=>['H', 'D'],'H'=>['E', 'M', 'K', 'G'],'J'=>['Z', 'R', 'I', 'L', 'U', 'N'],'W'=>['B'],'S'=>['B', 'P', 'I', 'G', 'O', 'Q', 'L', 'F', 'W', 'V'],'T'=>['K', 'F', 'I', 'B', 'V', 'Q', 'E'],'U'=>['W', 'O', 'V'],'L'=>['K', 'V', 'Q', 'U', 'F', 'C', 'Y'],'Y'=>[],'V'=>['M'])


In [138]:
preconditions = deepcopy(preconditions_save)

function get_ready_steps(preconditions)
    ready = Array{Char,1}()
    for pair in preconditions
        if isempty(pair[2])
            push!(ready, pair[1])
        end
    end
    # Find first step alphabetically
    sort!(ready)
    return ready
end

function take_action(preconditions, nextstep)
    delete!(preconditions, nextstep)
    for pair in preconditions
        filter!(x -> x != nextstep, pair[2])
    end
end

actions = Array{Char,1}()

while !isempty(preconditions)
    ready = get_ready_steps(preconditions)
    nextstep = ready[1]
    take_action(preconditions, nextstep)
    push!(actions, nextstep)
end
println(join(actions))

ABGKCMVWYDEHFOPQUILSTNZRJX


In [139]:
numworkers = 5
preconditions = deepcopy(preconditions_save)
actions = Array{Char,1}()
workers = Array{Any,1}(undef,numworkers)
fill!(workers, nothing)
timetaken = 0
while !isempty(preconditions) || any(workers.!==nothing)
    ready = get_ready_steps(preconditions)
    # Steps already being worked on
    underprogress = map(x -> x[1], workers[workers.!=nothing])
    # Assign to workers
    for step in ready
        if step in underprogress continue end
        k = findall(workers.==nothing)
        if !isempty(k)
            workers[k[1]] = [step, Int(step)-4]
        end
    end
    # Now either no more ready actions, or no free workers
    # Work to next action
    # Find next action to complete
    timesleft = map(x -> x[2], workers[workers.!=nothing])
    dt = minimum(timesleft)
    for w in workers[workers.!=nothing]
        w[2] -= dt
    end
    timetaken += dt
    for i in 1:length(workers)
        if workers[i]==nothing continue end
        w = workers[i]
        if w[2]==0
            take_action(preconditions, w[1])
            push!(actions, w[1])
            workers[i] = nothing
        end
    end
end
println(join(actions))
println(timetaken)

ABGYKMWCEVDHOFQUPILTSNZRJX
898
